##Add imports for datatype handling

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

##Mount the Azure container that holds the dataset

In [0]:
dbutils.fs.mount(
    source='wasbs://project-videogames-data@videogamedatasa50032904.blob.core.windows.net',
    mount_point='/mnt/project-videogames-data',
    extra_configs = {'fs.azure.account.key.videogamedatasa50032904.blob.core.windows.net':
        dbutils.secrets.get('projectvideogamesscope', 'storageAccountKey')}
    )

True

##Displaying contents of the containers

In [0]:
%fs
ls "/mnt/project-videogames-data"

path,name,size,modificationTime
dbfs:/mnt/project-videogames-data/raw-data/,raw-data/,0,0
dbfs:/mnt/project-videogames-data/transformed-data/,transformed-data/,0,0


##Read in the CSV into Databricks

In [0]:
vgsales = spark.read.format("csv").option("header","true").load("/mnt/project-videogames-data/raw-data/vgsales.csv")

##Display the data from the CSV

In [0]:
vgsales.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|          33|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

##

##Transformation - Changing data types

In [0]:
vgsales = vgsales.withColumn("Rank", col("Rank").cast(IntegerType()))
vgsales = vgsales.withColumn("Year", col("Year").cast(DateType()))
from pyspark.sql.functions import col, date_format
vgsales = vgsales.withColumn("Year", date_format(col("Year"), "yyyy"))
vgsales = vgsales.withColumn("NA_Sales", col("NA_Sales").cast(DoubleType()))
vgsales = vgsales.withColumn("EU_Sales", col("EU_Sales").cast(DoubleType()))
vgsales = vgsales.withColumn("JP_Sales", col("JP_Sales").cast(DoubleType()))
vgsales = vgsales.withColumn("Other_Sales", col("Other_Sales").cast(DoubleType()))
vgsales = vgsales.withColumn("Global_Sales", col("Global_Sales").cast(DoubleType()))

##Displaying new data types

In [0]:
vgsales.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



##Displaying top 100 items in order of Rank

In [0]:
rank_top_100 = vgsales.orderBy("Rank",ascending=True).limit(100).show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

##Displaying data where Publisher is "Nintendo"

In [0]:
publisher_nintendo = vgsales.filter(col("Publisher").contains("Nintendo")).limit(20).show()

+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing| Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports| Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing| Nintendo|   11.27|    8.89|   10.22|        1.0|       31.37|
|   6|              Tetris|      GB|1989|      Puzzle| Nintendo|

##Writing transformed data into Azure container

In [0]:
vgsales.write.option("header", 'true').csv("/mnt/project-videogames-data/transformed-data/vgsales")

##Creating a visualisation showing the most popular genres

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Year, Name, Genre, Global_Sales FROM vgsalestb")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Group by Genre against the sum of their Global Sales
grouped_df = pandas_df.groupby("Genre")["Global_Sales"].sum().reset_index()

#Sort by Global Sales
sorted_df = grouped_df.sort_values('Global_Sales', ascending=False)

# Create a custom color palette
custom_colors = [
    "#E6194B",  # Luminous Red
    "#3CB44B",  # Vivid Green
    "#FFE119",  # Bright Yellow
    "#0082C8",  # Strong Blue
    "#F58231",  # Vibrant Orange
    "#911EB4",  # Deep Purple
    "#46F0F0",  # Bright Cyan
    "#F032E6",  # Bright Magenta
    "#D2F53C",  # Lime Green
    "#FABEBE",  # Soft Pink
    "#008080",  # Teal
    "#E6BEFF"   # Lavender
]

# Create the bar chart using Plotly
fig = px.bar(sorted_df, x="Genre", y="Global_Sales", color= "Genre", color_discrete_sequence= custom_colors, title= "Most Popular Genres by Global Sales")
fig.update_layout(width=900, height=600, yaxis_title = "Global Sales / Million")  # Set plot size and change labels
fig.show()  # Display the plot


##Creating a visualisation showing most popular platforms

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Platform, Global_Sales FROM vgsalestb")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Group by Platform against the sum of their Global Sales
grouped_df = pandas_df.groupby("Platform")["Global_Sales"].sum().reset_index()

#Sort by Global Sales
sorted_df = grouped_df.sort_values('Global_Sales', ascending=False)

# Create a custom color palette
custom_colors = [
    "#E6194B",  # Luminous Red
    "#3CB44B",  # Vivid Green
    "#FFE119",  # Bright Yellow
    "#0082C8",  # Strong Blue
    "#F58231",  # Vibrant Orange
    "#911EB4",  # Deep Purple
    "#46F0F0",  # Bright Cyan
    "#F032E6",  # Bright Magenta
    "#D2F53C",  # Lime Green
    "#FABEBE",  # Soft Pink
    "#008080",  # Teal
    "#E6BEFF"   # Lavender
]

# Create the bar chart using Plotly
fig = px.bar(sorted_df, x="Platform", y="Global_Sales", color= "Platform", color_discrete_sequence= custom_colors, title="Most Popular Video Game Platforms by Global Sales")
fig.update_layout(width=900, height=600, yaxis_title = "Global Sales / Million")  # Set plot size and change labels
fig.show()  # Display the plot

##Creating a visualisation of the top 20 most popular games

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Name, Global_Sales FROM vgsalestb")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Group by Name against the sum of their Global Sales
grouped_df = pandas_df.groupby("Name")["Global_Sales"].sum().reset_index()

#Sort by Global Sales
sorted_df = grouped_df.sort_values('Global_Sales', ascending=False)

#Limit the displayed data to 20 items
shortened = sorted_df.head(20)

# Create a custom color palette
custom_colors = [
    "#E6194B",  # Luminous Red
    "#3CB44B",  # Vivid Green
    "#FFE119",  # Bright Yellow
    "#0082C8",  # Strong Blue
    "#F58231",  # Vibrant Orange
    "#911EB4",  # Deep Purple
    "#46F0F0",  # Bright Cyan
    "#F032E6",  # Bright Magenta
    "#D2F53C",  # Lime Green
    "#FABEBE",  # Soft Pink
    "#008080",  # Teal
    "#E6BEFF"   # Lavender
]

# Create the bar chart using Plotly
fig = px.bar(shortened, x="Name", y="Global_Sales", color= "Name", color_discrete_sequence= custom_colors, title="Top 20 Games by Global Sales")
fig.update_layout(width=900, height=600, yaxis_title = "Global Sales / Million")  # Set plot size and change labels
fig.show()  # Display the plot

##Creating a visualisation of the top 20 games by Rank

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Rank, Name, Global_Sales FROM vgsalestb Where Rank < 21")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Group by Name against Rank
grouped_df = pandas_df.groupby("Name")["Rank"].sum().reset_index()

#Sort by Global Sales
sorted_df = grouped_df.sort_values('Rank', ascending=True)

#Limit the displayed data to 20 items
#shortened = sorted_df.head(20)

# Create a custom color palette
custom_colors = [
    "#E6BEFF",  # Lavender
    "#008080",  # Teal
    "#FABEBE",  # Soft Pink
    "#D2F53C",  # Lime Green
    "#F032E6",  # Bright Magenta
    "#46F0F0",  # Bright Cyan
    "#911EB4",  # Deep Purple
    "#F58231",  # Vibrant Orange
    "#0082C8",  # Strong Blue
    "#FFE119",  # Bright Yellow
    "#3CB44B",  # Vivid Green
    "#E6194B"   # Luminous Red
]

# Create the bar chart using Plotly
fig = px.scatter(sorted_df, x="Name", y="Rank", color= "Name", color_discrete_sequence= custom_colors, title="Top 20 Games Ranked")
fig.update_layout(width=900, height=600, yaxis_title = "Rank")  # Set plot size and change labels
fig.update_yaxes(autorange="reversed")
fig.show()  # Display the plot

##Creating a visualisation of the most popular Genres by region

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Name, Genre, NA_Sales, EU_Sales, JP_Sales, Other_Sales FROM vgsalestb")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Aggregate sales data for each Region and Genre
df_aggregated = pandas_df.groupby('Genre').agg({'NA_Sales':'sum', 'EU_Sales':'sum', 'JP_Sales':'sum'}).reset_index()

# Combine Sales data for each region
df_melted = df_aggregated.melt(id_vars=['Genre'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales'], var_name='Region', value_name='Sales')

# Create bar chart
fig = px.bar(df_melted, x='Genre', y='Sales', color='Region', color_discrete_sequence= custom_colors, barmode='group', title='Most Popular Genres Per Region By Game Sales')
fig.update_layout(width=900, height=600, yaxis_title = "Region Sales / Million")  # Set plot size and change labels
fig.show() # Display the plot

##Creating a visualisation of the most popular Publishers

In [0]:
# Read the CSV file as a Spark DataFrame
vgsales = spark.read.format("csv") \
                  .option("header", "true") \
                  .option("inferSchema", "true") \
                  .load("/mnt/project-videogames-data/raw-data/vgsales.csv") \
                  .createOrReplaceTempView("temp_table")

# Create a Spark table from the temporary view - temp_table
spark.sql("CREATE TABLE IF NOT EXISTS vgsalestb USING parquet AS SELECT * FROM temp_table")

# Query for Global Sales
query_result = spark.sql("SELECT Publisher, Global_Sales FROM vgsalestb")

# Import Plotly Express for visualisation
import plotly.express as px

# Create a Pandas DataFrame for plotting.
pandas_df = query_result.toPandas()

# Group by Publisher against the sum of their Global Sales
grouped_df = pandas_df.groupby("Publisher")["Global_Sales"].sum().reset_index()

#Sort by Global Sales
sorted_df = grouped_df.sort_values('Global_Sales', ascending=False)

#Limit the displayed data to 20 items
shortened = sorted_df.head(20)

# Create a custom color palette
custom_colors = [
    "#E6194B",  # Luminous Red
    "#3CB44B",  # Vivid Green
    "#FFE119",  # Bright Yellow
    "#0082C8",  # Strong Blue
    "#F58231",  # Vibrant Orange
    "#911EB4",  # Deep Purple
    "#46F0F0",  # Bright Cyan
    "#F032E6",  # Bright Magenta
    "#D2F53C",  # Lime Green
    "#FABEBE",  # Soft Pink
    "#008080",  # Teal
    "#E6BEFF"   # Lavender
]

# Create the bar chart using Plotly
fig = px.bar(shortened, x="Publisher", y="Global_Sales", color= "Publisher", color_discrete_sequence= custom_colors, title="Most Popular Video Game Publishers by Global Sales")
fig.update_layout(width=900, height=600, yaxis_title = "Global Sales / Million")  # Set plot size and change labels
fig.show()  # Display the plot